# 17.3.2 가치 평가 클래스

In [4]:
import numpy as np
from valuation_class import valuation_class

In [5]:
class valuation_mcs_american(valuation_class):
    ''' 단일 요인 몬테카를로 시뮬레이션을 사용한 임의의 페이오프에 대한 아메리칸 옵션 가치 평가 클래스
    
    Method
    =======
    generate_payoff : 
        주어진 경로와 페이오프 함수를 이용하여 페이오프 계산
    present_value :
        롱스태프-슈바르츠 방식에 따른 현재 가치 반환
    
    '''
    
    def generate_payoff(self, fixed_seed = False):
        '''
        인수 
        ====
        fixed_seed : 
            가치 계산용 시드 값을 중복 사용
        
        '''
        try : 
            strike = self.strike
        
        except :
            pass
        
        paths = self.underlying.get_instrument_values(fixed_seed = fixed_seed)
        time_grid = self.underlying.time_grid
        
        try:
            time_index_start = int(np.where(time_grid == self.pricing_date)[0])
            time_index_end = int(np.where(time_grid == self.maturity)[0])
        
        except:
            print('Maturity date not in time grid of underlying')
        
        instrument_values = paths[time_index_start:time_index_end + 1]
        
        try:
            payoff = eval(self.payoff_func)
            return instrument_values, payoff, time_index_start, time_index_end
        
        except:
            print('Error evaluating payoff function')
    
    def present_value(self, accuracy = 6, fixed_seed = False, bf = 5, full = False):
        ''' 
        인수
        ====
        accuracy : int
            반환되는 결과의 자리수
        fixed_seed : Boolean
            가치 계산용 시드 값을 중복 사용
        bf : int
            회귀분석에 사용할 기저 함수의 숫자
        full : Boolean
            현재 가치의 1차원 배열 반환
        
        '''
        
        instrument_values, inner_values, time_index_start, time_index_end = self.generate_payoff(fixed_seed = fixed_seed)
        time_list = self.underlying.time_grid[time_index_start:time_index_end + 1]
        discount_factors = self.discount_curve.get_discount_factors(time_list, dtobjects = True)
        V = inner_values[-1]
        for t in range(len(time_list) - 2, 0, -1):
            # 주어진 시간 구간에 대한 할인율
            df = discount_factors[t, 1] / discount_factors[t + 1, 1]
            # 회귀분석 단계
            rg = np.polyfit(instrument_values[t], V * df, bf)
            # 경로에 대한 보유 가치 계산
            C = np.polyval(rg, instrument_values[t])
            # 최적 결정 다계
            # 내재 가치가 회귀분석한 보유 가치보다 크면 내재 가치 선택 그외엔 보유 가치 선택
            V = np.where(inner_values[t] > C, inner_values[t], V * df)
        
        df = discount_factors[0, 1] / discount_factors[1, 1]
        result = df * np.sum(V) / len(V)
        if full:
            return round(result, accuracy), df * V
        else:
            return round(result, accuracy)

# 17.3.3 사용 예

In [6]:
from dx_frame import *
from geometric_brownian_motion import geometric_brownian_motion

In [7]:
me_gbm = market_environment('me_gbm', dt.datetime(2015, 1, 1))
me_gbm.add_constant('initial_value', 36.)
me_gbm.add_constant('volatility', 0.2)
me_gbm.add_constant('final_date', dt.datetime(2016, 12, 31))
me_gbm.add_constant('currency', 'EUR')
me_gbm.add_constant('frequency', 'W')
me_gbm.add_constant('paths', 50000)

In [8]:
csr = constant_short_rate('csr', 0.06)

In [9]:
me_gbm.add_curve('discount_curve', csr)

In [10]:
gbm = geometric_brownian_motion('gbm', me_gbm)

In [11]:
# 아메리칸 풋 옵션의 Payoff
payoff_func = 'np.maximum(strike - instrument_values, 0)'

In [12]:
# 옵션의 초기 조건
# 만기 1년, 행사가 40
me_am_put = market_environment('me_am_put', dt.datetime(2015, 1, 1))
me_am_put.add_constant('maturity', dt.datetime(2015, 12, 31))
me_am_put.add_constant('strike', 40.)
me_am_put.add_constant('currency', 'EUR')

In [13]:
am_put = valuation_mcs_american('am_put', underlying = gbm, mar_env = me_am_put, payoff_func = payoff_func)

In [14]:
%time am_put.present_value(fixed_seed = True, bf = 5)

Wall time: 963 ms


4.64108

In [15]:
%%time
ls_table = []
for initial_value in (36, 38, 40, 42, 44):
    for volatility in (0.2, 0.4):
        for maturity in (dt.datetime(2015, 12, 31), dt.datetime(2016, 12, 31)):
            am_put.update(initial_value = initial_value,
                         volatility = volatility,
                         maturity = maturity)
            ls_table.append([initial_value,
                            volatility,
                            maturity,
                            am_put.present_value(bf = 5)])

Wall time: 20.5 s


In [16]:
print('S0 | Vola | T | Value')
print(22 * "-")
for r in ls_table:
    print('%d | %3.1f | %d | %5.3f' % (r[0], r[1], r[2].year - 2014, r[3]))

S0 | Vola | T | Value
----------------------
36 | 0.2 | 1 | 4.624
36 | 0.2 | 2 | 5.165
36 | 0.4 | 1 | 7.502
36 | 0.4 | 2 | 9.589
38 | 0.2 | 1 | 3.383
38 | 0.2 | 2 | 4.060
38 | 0.4 | 1 | 6.511
38 | 0.4 | 2 | 8.643
40 | 0.2 | 1 | 2.429
40 | 0.2 | 2 | 3.149
40 | 0.4 | 1 | 5.630
40 | 0.4 | 2 | 7.816
42 | 0.2 | 1 | 1.675
42 | 0.2 | 2 | 2.402
42 | 0.4 | 1 | 4.825
42 | 0.4 | 2 | 7.054
44 | 0.2 | 1 | 1.146
44 | 0.2 | 2 | 1.873
44 | 0.4 | 1 | 4.171
44 | 0.4 | 2 | 6.430


In [17]:
am_put.update(initial_value = 36)

In [18]:
am_put.delta()

-0.4831

In [19]:
am_put.vega()

22.6506